In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/MyDrive/CapStone')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# !ls /content/drive/MyDrive

In [3]:
import pandas as pd
import numpy as np
import json

In [4]:
try :
  import praw
except:
  !pip install praw
  import praw

In [23]:
try:
  from pyspark import SparkContext
  from pyspark.streaming import StreamingContext
  from pyspark.sql import SparkSession

except:
  !pip install pyspark
  from pyspark import SparkContext
  from pyspark.streaming import StreamingContext
  from pyspark.sql import SparkSession


In [6]:

reddit = praw.Reddit(client_id='P4lW9N9FiyiInyH3OEscaw',
                     client_secret='cCp4dL4ekj3oPz7pXu5P0cTDHbEf2w',
                     user_agent="web:com.example.myredditapp:v1.0.1 (by u/OneResearcher9936)")

subreddit = reddit.subreddit('AskReddit')
top_posts = subreddit.top(limit=100)

In [7]:
# top_posts = subreddit.top(limit=100)

In [31]:
subreddit = reddit.subreddit('AskReddit')
top_posts = subreddit.top(limit=100)

# Convert the data to a list of dictionaries
data = []
for post in top_posts:

    data.append({
        'title': post.title,
        'score': post.score,
        'num_comments': post.num_comments,
        'created_utc': post.created_utc,
        'permalink': post.permalink,
        'author': str(post.author)
    })

# Create a SparkSession
spark = SparkSession.builder.appName('RedditAnalysis').getOrCreate()


# df = spark.createDataFrame(data)

# Analyze the data using Spark SQL
# df.createOrReplaceTempView('posts')
spark.sql('SELECT author, AVG(score) as avg_score FROM posts GROUP BY author ORDER BY avg_score DESC LIMIT 10').show()


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



In [32]:
df = spark.createDataFrame(data)

In [34]:
df.createOrReplaceTempView('posts')

In [35]:
spark.sql('SELECT author, AVG(score) as avg_score FROM posts GROUP BY author ORDER BY avg_score DESC LIMIT 10').show()


+------------------+---------+
|            author|avg_score|
+------------------+---------+
|   ShoddySubstance| 221993.0|
|SoupIsAHotSmoothie| 217921.0|
|       txhorns1330| 197603.0|
|          esi_disi| 195913.0|
|              Ka1-| 186433.0|
|  Cheap_Double9726| 182987.0|
|          san69cor| 160333.0|
|       lauvnoodles| 154336.0|
|           slim_p_| 152116.0|
|      PastaM0nster| 148722.0|
+------------------+---------+

